# Acceso a datos de Google Cloud BigQuery en Python
Hay multiples forma de acceder a datos en BigQuery, y cada una tiene sus consideraciones

![img](bqpython.drawio.png)

In [13]:
sql = "SELECT 1 as a, 2 as b, 3 as c union all select 4,5,6"
project_id = "YOUR_GOOGLE_CLOUD_PROJECT_ID"

## a) BigQuery SDK
[https://cloud.google.com/python/docs/reference/bigquery/latest](https://cloud.google.com/python/docs/reference/bigquery/latest)

In [2]:
from google.cloud import bigquery

In [3]:
client = bigquery.Client()

In [4]:
# Dataframe python
df = client.query(sql).to_dataframe()

/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


In [5]:
type(df)

pandas.core.frame.DataFrame

In [6]:
df.head()

,a,b,c
0,1,2,3
1,4,5,6


In [7]:
# Forma alternativa
query_job = client.query(sql)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row)

Row((1, 2, 3), {'a': 0, 'b': 1, 'c': 2})
Row((4, 5, 6), {'a': 0, 'b': 1, 'c': 2})


#### Este calculo se realiza localmente en el notebook

In [8]:
df["a"].mean()

2.5

## b) Pandas-GBQ
[https://pandas-gbq.readthedocs.io/en/latest/reading.html](https://pandas-gbq.readthedocs.io/en/latest/reading.html)

In [9]:
!pip install pandas-gbq 

In [10]:
import pandas_gbq

In [14]:
df = pandas_gbq.read_gbq(sql, project_id=project_id)

/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


Downloading: 100%|██████████|


In [15]:
type(df)

pandas.core.frame.DataFrame

In [16]:
df.head()

,a,b,c
0,1,2,3
1,4,5,6


#### Este calculo se realiza localmente en el notebook

In [17]:
df["a"].mean()

2.5

## c) Using BigQuery BigFrames (Recomendado)
[https://cloud.google.com/bigquery/docs/dataframes-quickstart](https://cloud.google.com/bigquery/docs/dataframes-quickstart)

In [18]:
!pip install bigframes -U
!pip install --upgrade google-cloud-bigquery
!pip install numpy>=1.13 --upgrade

In [19]:
import bigframes.pandas as bpd
bpd.options.bigquery.project = project_id
bpd.options.bigquery.location = "us-central1"

In [20]:
df = bpd.read_gbq(sql)

In [21]:
type(df)

bigframes.dataframe.DataFrame

In [22]:
df.head()

,a,b,c
0,1,2,3
1,4,5,6


#### Este calculo se realiza automáticamente utilizando un job en BigQuery. Nota el query job id que se entrega después de los resultados

In [23]:
df["a"].mean()

2.5